In [44]:
from parsemrz import parse
from imagetomrz import ExtractData
import subprocess
import logging
import re
import json
import mrz
from mrz.checker.td3 import TD3CodeChecker
from mrz.checker.td2 import TD2CodeChecker
from mrz.checker.td1 import TD1CodeChecker

In [41]:
data = ExtractData('C:\\Users\\iyedn\\Downloads\\isabel-dos-santos-fake-passportjpg_49603.jpg')
data

['L<<00S<SANTOS<<<<<<<<<<<<<<<<<<<<<<<', 'WG08909176M18090600444B74<N01<1356']

In [45]:
TD3CodeChecker(data)

AttributeError: 'list' object has no attribute 'replace'

In [5]:
def parse(x):
    js_content = f"""
const parse = require('mrz').parse;

let mrz = {x};

var result = parse(mrz);
console.log(result);
"""

    with open('../mrz-parser/index.js', 'w') as js_file:
        js_file.write(js_content)

    try:
        # Run the Node.js command with 'cwd' set to the target directory
        node_command = ["node", "../mrz-parser", "index"]
        result = subprocess.run(node_command, check=True, text=True, capture_output=True)
        # Print any error output if present
        if result.stderr:
            logging.exception("Error output:")
            logging.exception(result.stderr)
        else:
            full =result.stdout
    except subprocess.CalledProcessError as e:
        logging.exception("Error running the command:", e)
    # Find the index of "fields:"
    fields_index = full.find("fields:")

    if fields_index != -1:
        # Find the index of the first "{" after "fields:"
        opening_brace_index = full.find("{", fields_index)

        # Find the index of the matching "}" for the first "{" after "fields:"
        closing_brace_index = opening_brace_index + 1
        brace_count = 1
        while brace_count > 0:
            if full[closing_brace_index] == "{":
                brace_count += 1
            elif full[closing_brace_index] == "}":
                brace_count -= 1
            closing_brace_index += 1

        extracted_text = full[opening_brace_index:closing_brace_index]
        extracted_text = re.sub(r'\x1b\[\d+m', '', extracted_text).strip()
        extracted_text = extracted_text.replace("  ","")
         
        pattern = re.compile(r'(\w+)\s*:\s*("[^"]*"|\S+)')
        extracted_text = pattern.sub(r'"\1":\2', extracted_text)
        extracted_text = extracted_text.replace("'","\"")
        print(extracted_text)
    else:
        print("Couldn't find 'fields:'")

In [33]:
tester = ['P<GBRTHATCHER<<ROSS<<<<<<<<<<<<<<<<<<<<<<<<<','2841464979TUN4904265M1601013<<<<<<<<<<<<<<04']

In [57]:
def cleanup(s: str) -> tuple:
    # Remove escape characters
    s = re.sub(r'\x1b\[[0-9;]*m', '', s)
    # Remove leading and trailing whitespaces
    s = s.strip()
    # Split the string into two parts
    parts = s.split(",\n")
    # Remove quotes and whitespaces from the first part
    part1 = parts[0].replace("'", "").strip()
    # Remove quotes and whitespaces from the second part
    part2 = parts[1].replace("'", "").strip()
    # Return the two parts as a tuple
    return (part1, part2)

In [58]:
import os
import re
import subprocess

def ExtractData(filepath):
    base_workdir = os.getcwd()
    try:
        # Run the Node.js command with 'cwd' set to the target directory
        node_command = ["node", "run/getMrz.js", "--file", filepath]
        result = subprocess.run(node_command, cwd=base_workdir, check=True)
        # Print any error output if present
        if result.stderr:
            logging.exception("Error output:")
            logging.exception(result.stderr)
    except subprocess.CalledProcessError as e:
        logging.exception("Error running the command:", e)
    
    # Get the file name from the file_location variable
    file_name = os.path.basename(filepath)

    # Split the file name and extension
    file_name, _ = os.path.splitext(file_name)

    # Change the file extension to .png
    file_name = file_name + ".png"

    # Access the /out/crop subdirectory
    new_dir = os.path.join(os.path.dirname(filepath), "out", "crop")

    # Join the new directory with the new file name
    filepath = os.path.join(new_dir, file_name)
    
    try:
        # Run the Node.js command with 'cwd' set to the target directory
        node_command = ["node", "run/readMrz.js", "--file", filepath]
        result = subprocess.run(node_command, cwd=base_workdir, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        print("Error running the command:", e)

    result_stdout = result.stdout # Updating variable name prevents potential problems with "."
    
    # Find the index of the first occurrence of 'mrz:'
    start_mrz_index = result_stdout.find('mrz:')

    if start_mrz_index != -1:
        # Find the index of the first '[' after 'mrz:'
        start_bracket_index = result_stdout.find('[', start_mrz_index)

        # Find the index of the first ']' after 'mrz:'
        end_bracket_index = result_stdout.find(']', start_mrz_index)

        if start_bracket_index != -1 and end_bracket_index != -1:
            # Extract the substring between the first '[' and the first ']' after 'mrz:'
            result = result_stdout[start_bracket_index + 1: end_bracket_index]

        else:
            print("No matching brackets found after 'mrz:'")
    else:
        print("No 'mrz:' found in the text.")
    result = cleanup(result)
    return result
st = ExtractData('C:\\Users\\iyedn\\OneDrive\\Desktop\\stage\\mrz-detection-master\\img\\210316052514-iceland-gender-neutral-passport.jpg')
st

('PAISLAEVARSDOTTIR<<THURIDUR<OESP<<<<<<<<<<<<',
 'A3536444<7ISL1212123<3103108121212<1239<<<68')

In [56]:
mrz.

('PAISLAEVARSDOTTIR<<THURIDUR<OESP<<<<<<<<<<<<',
 'A3536444<7ISL1212123<3103108121212<1239<<<68')

In [62]:
x = (int(input()))
k = (int(input()))

In [37]:
import re

def parse_td1_mrz(mrz_string: str):
    # Define the regular expression pattern for common TD documents
    pattern1 = r'([A|C|I][A-Z0-9<]{1})([A-Z]{3})([A-Z0-9<]{9})([0-9]{1})([A-Z0-9<]{15})\n([0-9]{6})([0-9]{1})([M|F|X|<]{1})([0-9]{6})([0-9]{1})([A-Z]{3})([A-Z0-9<]{11})([0-9]{1})\n([A-Z0-9<]{30})'
    pattern2 = r'([A|C|I][A-Z0-9<]{1})([A-Z]{3})([A-Z0-9<]{31})\n([A-Z0-9<]{9})([0-9]{1})([A-Z]{3})([0-9]{6})([0-9]{1})([M|F|X|<]{1})([0-9]{6})([0-9]{1})([A-Z0-9<]{7})([0-9]{1})'
    pattern3 = r'(P[A-Z0-9<]{1})([A-Z]{3})([A-Z0-9<]{39})\n([A-Z0-9<]{9})([0-9]{1})([A-Z0-9<]{3})([0-9]{6})([0-9]{1})([M|F|X|<]{1})([0-9]{6})([0-9]{1})([A-Z0-9<]{14})([0-9]{1})([A-Z0-9<]{14})'
    # Search for the pattern in the MRZ string
    match1 = re.match(pattern1, mrz_string)
    match2 = re.match(pattern2, mrz_string)
    match3 = re.match(pattern3, mrz_string)
    # Check if a match was found
    if match1:
        # Extract the matched groups
        groups = match1.groups()
        
        # Parse the groups
        result = {
            'document_type': groups[0].rstrip("<"),
            'issuing_country': groups[1],
            'document_number': groups[2],
            'check_digit_1': groups[3],
            'optional_data_1': groups[4],
            'date_of_birth': groups[5],
            'check_digit_2': groups[6],
            'sex': groups[7],
            'date_of_expiry': groups[8],
            'check_digit_3': groups[9],
            'nationality': groups[10],
            'optional_data_2': groups[11],
            'check_digit_4': groups[12],
            'names': groups[13]
        }
        
        return result
    elif match2:
        # Extract the matched groups
        groups = match2.groups()
        
        # Parse the groups
        result = {
            'document_type': groups[0].rstrip("<"),
            'issuing_country': groups[1],
            'primary_identifier': groups[2],
            'document_number': groups[3],
            'check_digit_1': groups[4],
            'nationality': groups[5],
            'date_of_birth': groups[6],
            'check_digit_2': groups[7],
            'sex': groups[8],
            'date_of_expiry': groups[9],
            'check_digit_3': groups[10],
            'optional_data': groups[11],
            'overall_check_digit': groups[12]
        }
        
        return result
    elif match3:
        # Extract the matched groups
        groups = match3.groups()
        
        # Parse the groups
        result = {
            'document_type': groups[0].rstrip("<"),
            'issuing_country': groups[1],
            'primary_identifier': groups[2],
            'document_number': groups[3],
            'check_digit_1': groups[4],
            'nationality': groups[5],
            'date_of_birth': groups[6],
            'check_digit_2': groups[7],
            'sex': groups[8],
            'date_of_expiry': groups[9],
            'check_digit_3': groups[10],
            'optional_data': groups[11],
            'overall_check_digit': groups[12]
        }
        
        return result
    else:
        # No match was found
        return None

# Example usage


In [36]:
def parse_td1_mrz(mrz_string: str):
    # Define the regular expression pattern for a TD1 document
    pattern = r'([A|C|I][A-Z0-9<]{1})([A-Z]{3})([A-Z0-9<]{9})([0-9]{1})([A-Z0-9<]{15})\n([0-9]{6})([0-9]{1})([M|F|X|<]{1})([0-9]{6})([0-9]{1})([A-Z]{3})([A-Z0-9<]{11})([0-9]{1})\n([A-Z0-9<]{30})'
    
    # Search for the pattern in the MRZ string
    match = re.search(pattern, mrz_string)
    
    # Check if a match was found
    if match:
        # Extract the matched groups
        groups = match.groups()
        
        # Parse the groups
        result = {
            'document_type': groups[0],
            'issuing_country': groups[1],
            'document_number': groups[2],
            'check_digit_1': groups[3],
            'optional_data_1': groups[4],
            'date_of_birth': groups[5],
            'check_digit_2': groups[6],
            'sex': groups[7],
            'date_of_expiry': groups[8],
            'check_digit_3': groups[9],
            'nationality': groups[10],
            'optional_data_2': groups[11],
            'check_digit_4': groups[12],
            'names': groups[13]
        }
        
        return result
    else:
        # No match was found
        return None

In [14]:
def parse_td2_mrz(mrz_string: str):
    # Define the regular expression pattern for a TD2 document
    pattern = r'([A|C|I][A-Z0-9<]{1})([A-Z]{3})([A-Z0-9<]{31})\n([A-Z0-9<]{9})([0-9]{1})([A-Z]{3})([0-9]{6})([0-9]{1})([M|F|X|<]{1})([0-9]{6})([0-9]{1})([A-Z0-9<]{7})([0-9]{1})'
    
    # Search for the pattern in the MRZ string
    match = re.search(pattern, mrz_string)
    
    # Check if a match was found
    if match:
        # Extract the matched groups
        groups = match.groups()
        
        # Parse the groups
        result = {
            'document_type': groups[0],
            'issuing_country': groups[1],
            'primary_identifier': groups[2],
            'document_number': groups[3],
            'check_digit_1': groups[4],
            'nationality': groups[5],
            'date_of_birth': groups[6],
            'check_digit_2': groups[7],
            'sex': groups[8],
            'date_of_expiry': groups[9],
            'check_digit_3': groups[10],
            'optional_data': groups[11],
            'overall_check_digit': groups[12]
        }
        
        return result
    else:
        # No match was found
        return None

In [19]:
def parse_td3_mrz(mrz_string: str):
    # Define the regular expression pattern for a TD3 document
    pattern = r'(P[A-Z0-9<]{1})([A-Z]{3})([A-Z0-9<]{39})\n([A-Z0-9<]{9})([0-9]{1})([A-Z]{3})([0-9]{6})([0-9]{1})([M|F|X|<]{1})([0-9]{6})([0-9]{1})([A-Z0-9<]{14})([0-9]{1})([0-9]{1})'
    # Search for the pattern in the MRZ string
    match = re.search(pattern, mrz_string)
    
    # Check if a match was found
    if match:
        # Extract the matched groups
        groups = match.groups()
        
        # Parse the groups
        result = {
            'document_type': groups[0],
            'issuing_country': groups[1],
            'primary_identifier': groups[2],
            'document_number': groups[3],
            'check_digit_1': groups[4],
            'nationality': groups[5],
            'date_of_birth': groups[6],
            'check_digit_2': groups[7],
            'sex': groups[8],
            'date_of_expiry': groups[9],
            'check_digit_3': groups[10],
            'optional_data': groups[11],
            'overall_check_digit': groups[12]
        }
        
        return result
    else:
        # No match was found
        return None

In [40]:
mrz_string = 'P<GBRBLUNDBY<<SPENCER<<<<<<<<<<<<<<<<<<<<<<<\n''0335189562GBR8306199<1601013<<<<<<<<<<<<<<02'
result = parse_td2_mrz(mrz_string)
print(result)

None


In [41]:
import re

def parse_td3_mrz(mrz_string: str):
    # Define the regular expression pattern for a TD3 document
    pattern = r'^(P[A-Z0-9<]{0,1})([A-Z]{3})([A-Z0-9<]{39})\n([A-Z0-9<]{9})([0-9]{1})([A-Z0-9<]{3})([0-9]{6})([0-9]{1})([M|F|X|<]{1})([0-9]{6})([0-9]{1})([A-Z0-9<]{14})([0-9]{1})([A-Z0-9<]{14})'
    
    # Search for the pattern in the MRZ string
    match = re.search(pattern, mrz_string, flags=re.IGNORECASE)
    
    # Check if a match was found
    if match:
        # Extract the matched groups
        groups = match.groups()
        
        # Parse the groups
        result = {
            'document_type': groups[0],
            'issuing_country': groups[1],
            'primary_identifier': groups[2],
            'document_number': groups[3],
            'check_digit_1': groups[4],
            'nationality': groups[5],
            'date_of_birth': groups[6],
            'check_digit_2': groups[7],
            'sex': groups[8],
            'date_of_expiry': groups[9],
            'check_digit_3': groups[10],
            'optional_data': groups[11],
            'overall_check_digit': groups[12]
        }
        
        return result
    else:
        # No match was found
        return None

# Example usage
mrz_string = 'P<GBRSMITH<<JOHN<<<<<<<<<<<<<<<<<<<<<<<<<<<<<\n1234567890GBR9001018M2301018<<<<<<<<<<<<<<08'
result = parse_td3_mrz(mrz_string)
print(result)


None


In [34]:
from mrz.checker.td1 import TD1CodeChecker, get_country
from mrz.checker.td3 import TD3CodeChecker
from mrz.checker.td2 import TD2CodeChecker
mrz_string = "P<UKRTKACHENKO<<MARIANA<<<<<<<<<<<<<<<<<<<<<XX000000<0UKR9108242F23092571234567890<<<<70"
if bool(TD1CodeChecker(mrz_string)):
    fields = td1_check.fields()
elif bool(TD2CodeChecker(mrz_string)):
    fields = td2_check.fields()
elif bool(TD3CodeChecker(mrz_string)):
    fields = td3_check.fields()

print(fields.sex, fields.surname)
print(get_country(fields.country))

LengthError: ('String was not recognized as a valid TD1. It should have 92 characters', '88')

In [1]:
from imagetomrz import ExtractData
from parsemrz import parse

In [2]:
list = ExtractData("C:\\Users\\iyedn\\OneDrive\\Desktop\\stage\\mrz-detection-master\\img\\210316052514-iceland-gender-neutral-passport.jpg")
l = parse(list)

In [7]:
print(l[6])

121212
